<h1>Read the data in</h1>

<p>Read data, do any preprocessing as necessary</p>

In [77]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv("./final_data.csv")

In [78]:
data.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,weather_main_mist,weather_main_rain,weather_main_thunderstorm,time_of_day_day,time_of_day_morning,time_of_day_night,season_fall,season_spring,season_summer,season_winter
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1


In [79]:
data.tail()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,weather_main_mist,weather_main_rain,weather_main_thunderstorm,time_of_day_day,time_of_day_morning,time_of_day_night,season_fall,season_spring,season_summer,season_winter
35012,2018-12-31 19:00:00+01:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
35013,2018-12-31 20:00:00+01:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
35014,2018-12-31 21:00:00+01:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
35015,2018-12-31 22:00:00+01:00,293.0,0.0,0.0,6562.0,2293.0,163.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
35016,2018-12-31 23:00:00+01:00,290.0,0.0,0.0,6926.0,2166.0,163.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1


In [80]:
data.columns

Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind onshore day ahead', 'total load forecast',
       'total load actual', 'price day ahead', 'price actual', 'temp',
       'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed',
       'wind_deg', 'rain_1h', 'rain_3h', 'snow_3h', 'clouds_all',
       'weather_main_clear', 'weather_main_

In [81]:
# Function that will sum some columns, average others
from sklearn.pipeline import Pipeline
from data_processing import *

def sum_avg(df, sum_cols, avg_cols, no_touch, period="D", dates_col="time"):
    # Get two dataframes, one to sum the other to average
    df_sum = df[sum_cols + [dates_col]].copy()
    df_avg = df[avg_cols + [dates_col]].copy()
    df_no_touch = df[no_touch + [dates_col]].copy()

    # Create pipelines needed
    avg_pipeline = Pipeline([
        ("avg", StatsPeriod(period=period, dates_col=dates_col))
    ])

    sum_pipeline = Pipeline([
        ("sum", StatsPeriod(aggregate="sum", period=period, dates_col=dates_col))
    ])

    untouched_pipeline = Pipeline([
        ("untouched", StatsPeriod(aggregate="", period=period, dates_col=dates_col))
    ])

    avg = avg_pipeline.fit_transform(df_avg)
    total = sum_pipeline.fit_transform(df_sum)
    untouched = untouched_pipeline.fit_transform(df_no_touch)
    return avg[0].join(total[0]).join(untouched[0]).copy()

In [82]:
# Get columns to sum over, average over
sum_vals = ['generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'rain_1h', 'rain_3h', 'snow_3h', 'clouds_all']

avg_vals = ['forecast solar day ahead',
       'forecast wind onshore day ahead', 'total load forecast',
       'total load actual', 'price day ahead', 'price actual', 'temp',
       'pressure', 'humidity', 'wind_speed',
       'wind_deg']

no_touch = ['temp_min', 'temp_max', 'weather_main_clear', 'weather_main_clouds', 'weather_main_drizzle',
       'weather_main_fog', 'weather_main_mist', 'weather_main_rain',
       'weather_main_thunderstorm', 'time_of_day_day', 'time_of_day_morning',
       'time_of_day_night', 'season_fall', 'season_spring', 'season_summer',
       'season_winter']

In [83]:
data = sum_avg(data, sum_vals, avg_vals, no_touch)

In [84]:
data.drop("price day ahead", axis=1, inplace=True)
data

,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price actual,temp,pressure,humidity,wind_speed,wind_deg,...,weather_main_mist,weather_main_rain,weather_main_thunderstorm,time_of_day_day,time_of_day_morning,time_of_day_night,season_fall,season_spring,season_summer,season_winter
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-31,17.000000,6436.000000,26118.000000,25385.000000,65.410000,272.491463,1016.400000,82.400000,2.000000,135.200000,...,False,False,False,False,False,True,False,False,False,True
2015-01-01,1049.500000,3942.416667,24753.250000,23966.958333,62.090833,277.001207,1022.125000,75.466667,1.933333,184.866667,...,False,False,False,False,False,False,False,False,False,True
2015-01-02,1187.208333,11117.041667,27519.416667,27188.541667,69.443750,277.951217,1024.691667,77.891667,1.575000,213.858333,...,False,False,False,False,False,False,False,False,False,True
2015-01-03,1133.375000,9113.416667,25500.833333,25097.750000,65.223333,278.829727,1020.375000,78.016667,1.558333,188.366667,...,False,False,False,False,False,False,False,False,False,True
2015-01-04,2292.041667,8022.833333,27167.875000,27104.916667,58.912083,279.643354,1016.008333,77.358333,1.250000,158.775000,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,707.375000,1300.083333,28856.500000,28551.708333,72.959583,280.766833,1027.175000,81.241667,1.525000,176.125000,...,False,False,False,False,False,False,False,False,False,True
2018-12-28,743.958333,4226.208333,28553.416667,28661.166667,69.230417,281.112083,1027.983333,82.175000,1.983333,174.783333,...,False,False,False,False,False,False,False,False,False,True
2018-12-29,673.333333,6665.833333,26427.166667,26603.958333,62.623333,281.105917,1029.400000,78.625000,2.833333,176.533333,...,False,False,False,False,False,False,False,False,False,True


<h1>Run the model</h1>
<p>Predict Price and Load based on dataset</p>

In [85]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[data.columns[~data.columns.isin(["price actual"])]], data["price actual"], test_size=0.2)

In [86]:
# View Training Set
X_train

,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,temp,pressure,humidity,wind_speed,wind_deg,generation biomass,...,weather_main_mist,weather_main_rain,weather_main_thunderstorm,time_of_day_day,time_of_day_morning,time_of_day_night,season_fall,season_spring,season_summer,season_winter
Date,,,,,,,,,,,,,,,,,,,,,
2018-06-13,2236.958333,7011.375000,28839.875000,28904.541667,293.795083,1016.541667,57.200000,3.133333,215.675000,7580.0,...,False,False,False,False,False,False,False,False,True,False
2016-01-04,2121.250000,6459.208333,29326.916667,29071.375000,286.474238,1005.225000,77.375000,5.758333,224.266667,10673.0,...,False,False,False,False,False,False,False,False,False,True
2015-08-20,2217.000000,2674.791667,28033.625000,28708.833333,299.142250,1018.800000,58.075000,2.550000,166.133333,13272.0,...,False,False,False,False,False,False,False,False,True,False
2016-07-30,2162.291667,4895.458333,28864.916667,28514.500000,300.369500,1015.133333,63.566667,2.866667,174.016667,9336.0,...,False,False,False,False,False,False,False,False,True,False
2016-09-20,1641.791667,3687.166667,28710.458333,28818.791667,293.043400,1014.000000,67.000000,1.541667,177.758333,8178.0,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-09-29,1781.666667,4075.916667,29094.333333,28689.958333,293.888083,1022.016667,62.533333,1.933333,154.925000,8523.0,...,False,False,False,False,False,False,True,False,False,False
2015-05-18,2285.458333,4814.583333,27857.833333,28167.041667,292.045431,998.966667,60.100000,2.466667,197.858333,11964.0,...,False,False,False,False,False,False,False,True,False,False
2015-12-16,623.083333,5408.666667,30776.625000,30672.000000,285.956742,1029.575000,67.816667,1.883333,177.108333,12177.0,...,False,False,False,False,False,False,False,False,False,True


In [87]:
y_train

Date
2018-06-13    59.475000
2016-01-04    42.022500
2015-08-20    66.120000
2016-07-30    46.073750
2016-09-20    50.580833
                ...    
2016-09-29    50.357917
2015-05-18    62.684167
2015-12-16    64.761250
2015-01-22    64.075417
2016-06-06    50.851250
Freq: D, Name: price actual, Length: 1169, dtype: float64

In [128]:
# Train the model
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, r2_score

model = XGBRegressor(random_state=42, max_depth=10, learning_rate=0.01, n_estimators=800)

In [140]:
neg_mean_square_cv = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
neg_mean_square_cv

array([-40.49842334, -41.33233655, -47.14787403, -50.28147861,
       -44.384778  ])

In [141]:
r2_cv = cross_val_score(model, X_train, y_train, cv=5, scoring="r2")
r2_cv

array([0.74255685, 0.69473436, 0.66156103, 0.64983992, 0.67544146])

In [142]:
# Fit and train the model
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=800, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [143]:
# See the performance of the model on the training set
predict_train = model.predict(X_train)
predict_train

array([59.495552, 42.066746, 66.48844 , ..., 64.04553 , 64.29324 ,
       50.99129 ], dtype=float32)

In [144]:
# Determine MSE
mean_squared_error(y_train, predict_train)

0.24222116380218967

In [145]:
# Determine R2
r2_score(y_train, predict_train)

0.9983053442513455

In [146]:
# Determine performance on test set
predict = model.predict(X_test)

In [147]:
# Determine MSE
mean_squared_error(y_test, predict)

38.74106291564759

In [148]:
# Determine R2
r2_score(y_test, predict)

0.7499026380794235

<h1>Total Load Predictions</h1>

<p>Predict total load values based on dataset</p>